# Azure Machine Learning SDK 
## 목차
> **1. Data Load**
    - Azure Machine Learning DataSet으로 부터 
    - Directly File Share로 부터
**2. Data Join**
    - Pandas
**3. Data Export**
    - Azure Machine Learning DataSet으로
    - Directly File Share로  
**4. Autoamted ML**
    - Model 등록하기 
**5. Result**
    - 결과 공유하기

# 0. Set Up
- Azure Machine Learning SDK에서 제공해주는 azureml 모듈을 Import 합니다.

In [31]:
import pandas as pd
import azureml.core
import pandas as pd
import numpy as np
import logging

from azureml.core import Workspace, Experiment, Dataset
from azureml.train.automl import AutoMLConfig
from datetime import datetime

- 현재 사용하고 있는 Azure Machine Learning Workspace를 코드 상에서 Connection합니다.

In [33]:
ws = Workspace.from_config()

experiment_name = 'automl-azuremlSDK'

experiment = Experiment(ws, experiment_name)

output = {}
output['SDK version'] = azureml.core.VERSION
output['Subscription ID'] = ws.subscription_id
output['Workspace'] = ws.name
output['SKU'] = ws.sku
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
output['Run History Name'] = experiment_name
pd.set_option('display.max_colwidth', -1)
outputDf = pd.DataFrame(data = output, index = [''])
outputDf.T

,
SDK version,1.0.76
Subscription ID,81e13e1b-5d14-4a77-9b50-0803f4705d12
Workspace,hw-cloocus-mlsrv
SKU,Enterprise
Resource Group,rg_poc_hw02
Location,koreacentral
Run History Name,automl-azuremlSDK


# 1. Data Load

### 1) Azure Machine Learning DataSet으로 부터
- Azure ML에 등록된 Dataset을 쉽게 Import 할 수 있습니다.

In [3]:
# azureml-core 버전 1.0.72 이상이 필요함
#################################### 아래 코드 변경 필요 #######################################
from azureml.core import Workspace, Dataset

subscription_id = '81e13e1b-5d14-4a77-9b50-0803f4705d12'
resource_group = 'RG_POC_HW02'
workspace_name = 'hw-cloocus-mlsrv'

workspace = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_name(workspace, name='ibpX')

###############################################################################################


xdataset = dataset.to_pandas_dataframe()

In [13]:
xdataset.head()

,timestamp,A2FIC1001,A2FI1016,A2TI1039,A2PIC1019,A2TIC1045,A2FIC1023,A2TI1260,A2TIC1040,A2FIC1027,A2FIC1020,A2PI1258,A2PDI1017A,A2PDI1017B,A2PI1018,A2TI1204
0,201801011500,525.0740,519.4849,71.44790,0.679275,44.53076,189.7352,29.24771,86.22105,2.180395,211.3417,1.288046,0.607279,0.195648,1.483658,120.5992
1,201801021500,524.4335,523.7094,72.50761,0.681772,45.61728,185.3640,28.21494,86.84060,2.075200,209.7350,1.250542,0.571002,0.223391,1.475044,121.2669
2,201801031500,526.8066,524.0200,72.36311,0.681633,44.57035,189.4110,27.91968,86.76717,2.675530,206.6377,1.273913,0.593844,0.210924,1.484559,121.0310
3,201801041500,525.6947,527.2949,72.79662,0.682682,44.76646,188.7758,27.82124,87.42059,2.127430,208.3724,1.264119,0.586615,0.216437,1.485460,121.4138
4,201801051500,524.6626,523.5300,73.32645,0.678926,49.85023,189.7892,28.85439,87.66037,2.332066,207.7811,1.286043,0.601292,0.198154,1.481454,121.6497


In [4]:
# azureml-core 버전 1.0.72 이상이 필요함
#################################### 아래 코드 변경 필요 #######################################
from azureml.core import Workspace, Dataset

subscription_id = '81e13e1b-5d14-4a77-9b50-0803f4705d12'
resource_group = 'RG_POC_HW02'
workspace_name = 'hw-cloocus-mlsrv'

workspace = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_name(workspace, name='ibpY')
###############################################################################################


ydataset = dataset.to_pandas_dataframe()

In [6]:
ydataset.head()

,timestamp,IBP
0,201801011500,96.3
1,201801021500,96.8
2,201801031500,96.2
3,201801041500,96.8
4,201801051500,97.9



### 2) JMP (File Share)으로 부터
- Jupyter Notebook VM에 Terminal 에서 작업을 통해 연동 가능합니다.

# 2. Data Join
- Python Code로 Timestamp 기준으로 데이터를 Join합니다.
- pandas에 merge() 사용

In [26]:
print("xdata의 row 수 : {}, xdata의 column 수 : {}".format(xdataset.shape[0], xdataset.shape[1]))
print("xdata의 row 수 : {}, xdata의 column 수 : {}".format(ydataset.shape[0], ydataset.shape[1]))

xdata의 row 수 : 439, xdata의 column 수 : 16
xdata의 row 수 : 439, xdata의 column 수 : 2


In [9]:
fulldata = pd.merge(xdataset, ydataset)

fulldata.head()

,timestamp,A2FIC1001,A2FI1016,A2TI1039,A2PIC1019,A2TIC1045,A2FIC1023,A2TI1260,A2TIC1040,A2FIC1027,A2FIC1020,A2PI1258,A2PDI1017A,A2PDI1017B,A2PI1018,A2TI1204,IBP
0,201801011500,525.0740,519.4849,71.44790,0.679275,44.53076,189.7352,29.24771,86.22105,2.180395,211.3417,1.288046,0.607279,0.195648,1.483658,120.5992,96.3
1,201801021500,524.4335,523.7094,72.50761,0.681772,45.61728,185.3640,28.21494,86.84060,2.075200,209.7350,1.250542,0.571002,0.223391,1.475044,121.2669,96.8
2,201801031500,526.8066,524.0200,72.36311,0.681633,44.57035,189.4110,27.91968,86.76717,2.675530,206.6377,1.273913,0.593844,0.210924,1.484559,121.0310,96.2
3,201801041500,525.6947,527.2949,72.79662,0.682682,44.76646,188.7758,27.82124,87.42059,2.127430,208.3724,1.264119,0.586615,0.216437,1.485460,121.4138,96.8
4,201801051500,524.6626,523.5300,73.32645,0.678926,49.85023,189.7892,28.85439,87.66037,2.332066,207.7811,1.286043,0.601292,0.198154,1.481454,121.6497,97.9


In [29]:
print("fulldata row 수 : {}, fulldata column 수 : {}".format(fulldata.shape[0], fulldata.shape[1]))

fulldata row 수 : 439, fulldata column 수 : 17


# 3. Export Data

In [53]:
fulldata.to_csv('./fulldata_ibp.csv', index=False, header=True)

In [56]:
from azureml.core import Workspace, Datastore, Dataset

datastore = Datastore.get(ws, "datastore")
datastore.upload_files(files = ['./fulldata_ibp.csv'], target_path = 'fulldata_ibp/', overwrite = True,show_progress = True)

Uploading an estimated of 1 files
Uploading ./fulldata_ibp.csv
Uploaded ./fulldata_ibp.csv, 1 files out of an estimated total of 1
Uploaded 1 files


$AZUREML_DATAREFERENCE_58d443cbb59142418832bd9763f0b786

# 4. Autoamted ML

### 1) Train Compute 지정

In [37]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget

amlcompute_cluster_name = "HWHOLTrainingCl"

found=False

cts = ws.compute_targets

if amlcompute_cluster_name in cts and cts[amlcompute_cluster_name].type == 'AmlCompute':
    found = True
    print('Found existing compute target.')
    compute_target = cts[amlcompute_cluster_name]
    
if not found:
    print('Creating a new compute target...')
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = "STANDARD_D4_V2",max_nodes = 4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, provisioning_config)

print('Checking cluster status...')

compute_target.wait_for_completion(show_output = True, min_node_count = None, timeout_in_minutes = 20)

Found existing compute target.
Checking cluster status...
Succeeded
AmlCompute wait for completion finished
Minimum number of nodes requested have been provisioned


### 2) Train & Test 데이터 셋 Split

In [59]:
dataset = Dataset.Tabular.from_delimited_files(path=datastore.path('fulldata_ibp/fulldata_ibp.csv'))

In [60]:
train_data, test_data = dataset.random_split(percentage=0.8, seed=223)

### 3) Train & Test 데이터 셋 저장하기

In [61]:
# train 데이터 셋 등록하기 
train_data.register(workspace = ws, name = 'IBP_train_dataset',
                       description = 'IBP Prediction by Automated ML',
                      create_new_version=True)

# test 데이터 셋 등록하기
test_data.register(workspace = ws, name = 'IBP_test_dataset', description = 'IBP Prediction by Automated ML', create_new_version=True)

train_data.to_pandas_dataframe().head()

,timestamp,A2FIC1001,A2FI1016,A2TI1039,A2PIC1019,A2TIC1045,A2FIC1023,A2TI1260,A2TIC1040,A2FIC1027,A2FIC1020,A2PI1258,A2PDI1017A,A2PDI1017B,A2PI1018,A2TI1204,IBP
0,201801011500,525.07,519.48,71.45,0.68,44.53,189.74,29.25,86.22,2.18,211.34,1.29,0.61,0.20,1.48,120.60,96.30
1,201801021500,524.43,523.71,72.51,0.68,45.62,185.36,28.21,86.84,2.08,209.74,1.25,0.57,0.22,1.48,121.27,96.80
2,201801031500,526.81,524.02,72.36,0.68,44.57,189.41,27.92,86.77,2.68,206.64,1.27,0.59,0.21,1.48,121.03,96.20
3,201801051500,524.66,523.53,73.33,0.68,49.85,189.79,28.85,87.66,2.33,207.78,1.29,0.60,0.20,1.48,121.65,97.90
4,201801081500,464.64,474.04,69.14,0.68,51.17,196.38,29.10,81.78,2.19,161.33,1.20,0.52,0.18,1.39,121.15,98.00


### 4) Automated ML 셋팅하기

In [63]:
automl_settings = {
    "enable_early_stopping": True, 
    "experiment_timeout_hours" : 0.2,
    "max_concurrent_iterations": 4,
    "max_cores_per_iteration": -1,
    "n_cross_validations": 5,
    "primary_metric": 'normalized_root_mean_squared_error',
    "verbosity": logging.INFO
}

# 예측할 필드 - IBP### 3) Train & Test 데이터 셋 저장하기
label ="IBP"

automl_config = AutoMLConfig(task = 'regression',
                             debug_log = 'automl_errors.log',
                             compute_target=compute_target,
                             training_data = train_data,
                             label_column_name = label,
                             **automl_settings
                            )

In [64]:
remote_run = experiment.submit(automl_config, show_output = False)

WARNING - Received unrecognized parameter: experiment_timeout_hours 0.2


In [65]:
remote_run

Experiment,Id,Type,Status,Details Page,Docs Page
automl-azuremlSDK,AutoML_04bdcc36-3b24-460e-bb42-2b4b7863ffe0,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


In [ ]:
remote_run.wait_for_completion()

In [ ]:
best_run, fitted_model = remote_run.get_output()

In [ ]:
best_run_customized, fitted_model_customized = remote_run.get_output()

In [ ]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()